In [1]:
import open3d as o3d
import numpy as np
from copy import deepcopy
import copy

import mcap
import rosbag2_py
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
from sensor_msgs.msg import PointCloud2, PointField
import math

import sys
from collections import namedtuple
import struct


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Util funcs for PCD Voxalization and noise

In [2]:
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0,
                                             max_nn=30))
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5.0,
                                             max_nn=100),
    )
    return (pcd_down, pcd_fpfh)

def apply_noise(pcd, mu, sigma):
    noisy_pcd = copy.deepcopy(pcd)
    points = np.asarray(noisy_pcd.points)
    points += np.random.normal(mu, sigma, size=points.shape)
    noisy_pcd.points = o3d.utility.Vector3dVector(points)
    return noisy_pcd


### Reading PCD

In [3]:
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

print("Reading inputs")

src = o3d.io.read_point_cloud("/home/aditya/Documents/Personal/jb/interview/dexory_ex/back.pcd")
dst = o3d.io.read_point_cloud("/home/aditya/Documents/Personal/jb/interview/dexory_ex/front.pcd")


Reading inputs
[Open3D DEBUG] Format auto File /home/aditya/Documents/Personal/jb/interview/dexory_ex/back.pcd
[Open3D DEBUG] PCD header indicates 3 fields, 12 bytes per point, and 57632 points in total.
[Open3D DEBUG] x, F, 4, 1, 0
[Open3D DEBUG] y, F, 4, 1, 4
[Open3D DEBUG] z, F, 4, 1, 8
[Open3D DEBUG] Compression method is 1.
[Open3D DEBUG] Points: yes;  normals: no;  colors: no
[Open3D DEBUG] Read geometry::PointCloud: 57632 vertices.
[Open3D DEBUG] Format auto File /home/aditya/Documents/Personal/jb/interview/dexory_ex/front.pcd
[Open3D DEBUG] PCD header indicates 3 fields, 12 bytes per point, and 57632 points in total.
[Open3D DEBUG] x, F, 4, 1, 0
[Open3D DEBUG] y, F, 4, 1, 4
[Open3D DEBUG] z, F, 4, 1, 8
[Open3D DEBUG] Compression method is 1.
[Open3D DEBUG] Points: yes;  normals: no;  colors: no
[Open3D DEBUG] Read geometry::PointCloud: 57632 vertices.


### Parameters for RANSAC and ICP

In [4]:
voxel_size = 0.5
distance_threshold = 1.5 * voxel_size
method = "from_features"
mutual_filter=False
max_iterations = 10000000
confidence = 0.99999

### Down Sampling using Voxelization

In [5]:
print("Downsampling inputs")
src_down, src_fpfh = preprocess_point_cloud(src, voxel_size)
dst_down, dst_fpfh = preprocess_point_cloud(dst, voxel_size)

Downsampling inputs
[Open3D DEBUG] Pointcloud down sampled from 57632 points to 5004 points.
[Open3D DEBUG] Pointcloud down sampled from 57632 points to 5671 points.


## RANSAC - Global Registration
### Given 2 PCD - Estimate loosly estimated Transformation using Feature Correpondence

In [6]:
if method == "from_features":
    print("Running RANSAC from features")
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        src_down,
        dst_down,
        src_fpfh,
        dst_fpfh,
        mutual_filter=mutual_filter,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.
        TransformationEstimationPointToPoint(False),
        ransac_n=3,
        checkers=[
            o3d.pipelines.registration.
            CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold),
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(
            max_iterations, confidence),
    )
    
#     visualize_registration(src, dst, result.transformation)
    print(result.transformation)

elif method == "from_correspondences":
    print("Running RANSAC from correspondences")
    # Mimic importing customized external features (e.g. learned FCGF features) in numpy
    # shape: (feature_dim, num_features)
    src_fpfh_np = np.asarray(src_fpfh.data).copy()
    dst_fpfh_np = np.asarray(dst_fpfh.data).copy()

    src_fpfh_import = o3d.pipelines.registration.Feature()
    src_fpfh_import.data = src_fpfh_np

    dst_fpfh_import = o3d.pipelines.registration.Feature()
    dst_fpfh_import.data = dst_fpfh_np

    corres = o3d.pipelines.registration.correspondences_from_features(
        src_fpfh_import, dst_fpfh_import, mutual_filter)
    result = o3d.pipelines.registration.registration_ransac_based_on_correspondence(
        src_down,
        dst_down,
        corres,
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.
        TransformationEstimationPointToPoint(False),
        ransac_n=3,
        checkers=[
            o3d.pipelines.registration.
            CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold),
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(
            max_iterations, confidence),
    )
    # visualize_registration(src, dst, result.transformation)
    print(result.transformation)

Running RANSAC from features
[Open3D DEBUG] Thread 000013: registration fitness=0.054, corres inlier ratio=0.001, Est. max k = 6678575718.757569
[Open3D DEBUG] Thread 1666879: registration fitness=0.132, corres inlier ratio=0.002, Est. max k = 2817524085.4936843[Open3D DEBUG] Thread 5000088: registration fitness=0.131, corres inlier ratio=0.002, Est. max k = 1083825929.5169973

[Open3D DEBUG] Thread 7500396: registration fitness=0.078, corres inlier ratio=0.002, Est. max k = 1442572321.7517476
[Open3D DEBUG] Thread 4167001: registration fitness=0.048, corres inlier ratio=0.001, Est. max k = 11540577972.139704
[Open3D DEBUG] Thread 5833657: registration fitness=0.146, corres inlier ratio=0.002, Est. max k = 1083825929.5169973[Open3D DEBUG] Thread 9167179: registration fitness=0.132, corres inlier ratio=0.001, Est. max k = 6678575718.757569

[Open3D DEBUG] Thread 834170: registration fitness=0.202, corres inlier ratio=0.002, Est. max k = 1442572321.7517476
[Open3D DEBUG] Thread 6667251: 

### Out of RANSAC - Initial Estimation for Local Registration

In [15]:
print(
    "Displaying Downsampled source and target point cloud with initial transformation:", result.transformation
)

threshold = 1.0
voxel_size = 0.5
source, src_fpfh = preprocess_point_cloud(src, voxel_size)
target, dst_fpfh = preprocess_point_cloud(dst, voxel_size)

trans_init = result.transformation
# draw_registration_result(source, target, trans_init)

Displaying Downsampled source and target point cloud with initial transformation: [[-9.99866954e-01  1.03866834e-03  1.62786776e-02 -1.38153541e+00]
 [-1.01186736e-03 -9.99998119e-01  1.65453566e-03 -6.09135529e-02]
 [ 1.62803655e-02  1.63784367e-03  9.99866125e-01  5.86706620e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[Open3D DEBUG] Pointcloud down sampled from 57632 points to 5004 points.
[Open3D DEBUG] Pointcloud down sampled from 57632 points to 5671 points.


### Local Registration - Iterative Point Cloud (Point to Plane using Huber Kernel)

In [16]:

mu, sigma = 0, 0.1
source_noisy = apply_noise(source, mu, sigma)

icp_method = "p2l"

if icp_method=="p2l":
    print("Robust point-to-plane ICP, threshold={}:".format(threshold))
    loss = o3d.pipelines.registration.HuberLoss(k=sigma)
    print("Using robust loss:", loss)

    p2l = o3d.pipelines.registration.TransformationEstimationPointToPlane(loss)

    reg_p2l = o3d.pipelines.registration.registration_icp(
        source_noisy, target, threshold, trans_init, p2l)
    print(reg_p2l)
    print("Transformation is:")
    print(reg_p2l.transformation)

if icp_method=="p2p":
    print("Apply point-to-point ICP")
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source_noisy, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    print(reg_p2p)
    print("Transformation is:")
    print(reg_p2p.transformation)



Robust point-to-plane ICP, threshold=1.0:
Using robust loss: RobustKernel::HuberLoss with k=0.100000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.7760, RMSE 0.4275
[Open3D DEBUG] Residual : 5.53e-02 (# of elements : 3883)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.7752, RMSE 0.4197
[Open3D DEBUG] Residual : 5.16e-02 (# of elements : 3879)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.7786, RMSE 0.4198
[Open3D DEBUG] Residual : 5.16e-02 (# of elements : 3896)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.7804, RMSE 0.4200
[Open3D DEBUG] Residual : 5.26e-02 (# of elements : 3905)
[Open3D DEBUG] ICP Iteration #4: Fitness 0.7808, RMSE 0.4195
[Open3D DEBUG] Residual : 5.23e-02 (# of elements : 3907)
[Open3D DEBUG] ICP Iteration #5: Fitness 0.7812, RMSE 0.4197
[Open3D DEBUG] Residual : 5.25e-02 (# of elements : 3909)
[Open3D DEBUG] ICP Iteration #6: Fitness 0.7814, RMSE 0.4199
[Open3D DEBUG] Residual : 5.23e-02 (# of elements : 3910)
[Open3D DEBUG] ICP Iteration #7: Fitness 0.7808, RMSE 0.4192
[Open

### Final Transformation 

In [18]:
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)

RegistrationResult with fitness=7.807754e-01, inlier_rmse=4.193596e-01, and correspondence_set size of 3907
Access transformation to get result.
Transformation is:
[[-9.99960358e-01  1.03484041e-03  8.84371537e-03 -1.48305767e+00]
 [-1.01199871e-03 -9.99996142e-01  2.58690122e-03 -2.15588195e-01]
 [ 8.84635828e-03  2.57784884e-03  9.99957547e-01  3.13695749e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Tranlational and Rotational Error in Missalignment 

### Error in X-axis 
### - Data from tf_static = -1.5205 m 
### - Estimated using ICP = -1.48304 m

In [34]:
error_x = -1.5205 - (-1.48305767)
print("error_x:", error_x, " m")

error_x: -0.03744232999999997  m


### Note: The same 4 cm tranlational error reflects in visually in data

### Error in Y-axis 
### - Data from tf_static = -0.2828 m 
### - Estimated using ICP = -0.2155 m

In [31]:
error_y = -0.2828 - (-0.2155)
print("error_y:", error_y, " m")

error_y: -0.0673  m


### Note: The same 6 cm tranlational error reflects in visually in data

### Rotation Error

### rotation matrix = 
###                   [-9.99960358e-01 , 1.03484041e-03 ,  8.84371537e-03 ]
###                    [-1.01199871e-03, -9.99996142e-01 ,  2.58690122e-03 ]
###                    [ 8.84635828e-03,   2.57784884e-03 ,   9.99957547e-01 ]

In [35]:
estimated_rot = [ 0.3982231, 0.2324937, 180.0580482 ]

In [36]:
rot = [0, 0, 180]